In [56]:
#import OS to share the project dir
import os
os.chdir('F:\\fourth_year\Pattern_Classification\project\Writer_Identification') 

#import important packages
from IAM_loader import IAM_loader
#import tensorflow as tf
#import pandas as pd
import numpy as np
from sklearn import svm
from sklearn.neighbors import KNeighborsClassifier
import cv2 as cv2
from PIL import Image
import math
from skimage import feature
from operator import itemgetter
from scipy import stats
from sklearn.metrics import mean_squared_error
from skimage.feature import peak_local_max
from scipy.signal import find_peaks
from scipy.signal import argrelextrema
#packages for visualization
import matplotlib.pyplot as plt
import matplotlib.cm as cm
#allows charts to appear in the notebook 
%matplotlib inline

#ignoring warnings
import warnings
warnings.filterwarnings('ignore')

## Reading Data :

In [57]:
# we use IAM handwriting database download it first
# here we will read the images and get the important info about the images 
# we divide the available images into (train,cv and test)
# division will be depend on the number of the images 

training_data=[] # only the train 
training_data_threshold=[] #training black images 
test_data_threshold=[] #test black images 
validation_data=[] # cross validation 
test_data=[] # test images 

WIDTH=2175 #image width
HEIGHT=2304 #image height
path="../dataset/output2"


In [58]:
def reading_and_preprocess():
    loader=IAM_loader('../dataset')
    training_data,test_data,validation_data = loader.split_data()
    #writting the data 
    training_data_threshold=training_data
    test_data_threshold=validation_data
    
    #getting the thresholded images of the training
    for i in range(len(training_data)):
            temp=training_data[i][0]
            _,image= cv2.threshold(temp,0,255,cv2.THRESH_BINARY_INV)
            training_data_threshold[i][0]=image
            
    #getting the thresholded images of the test
    for i in range(len(validation_data)):
            temp=validation_data[i][0]
            _,image= cv2.threshold(temp,0,255,cv2.THRESH_BINARY_INV)
            test_data_threshold[i][0]=image
    return training_data,test_data,validation_data,training_data_threshold,test_data_threshold

## Preprocessing Module:

### Resize Image:

In [59]:
#get the window that contains the words 
def crop_image(image,y1,y2,x1,x2):
    image=image[y1:y2,x1:x2]
    return image

### Crop the images:

In [60]:
#note : use WIDTH and HEIGHT 
def resize_image (img):
    img=cv2.resize(img, dsize=(HEIGHT, WIDTH), interpolation=cv2.INTER_CUBIC)
    return img

### Detect lines to get the region of handwritten document: 

In [61]:
def detect_lines(img):
    horizontal_lines=[]
    ret_lines=[] 
    #apply canny 
    edges = cv2.Canny(img,50,150,apertureSize = 3)
    #dilation
    kernel = np.ones((5,5), np.uint8)
    img_dilation = cv2.dilate(edges, kernel, iterations=1)
    #hough line transform
    lines = cv2.HoughLinesP(img_dilation, 1, np.pi/180,200, maxLineGap=1,minLineLength=img.shape[1]/2)
    #getting the horizontal lines 
    for i in range(len(lines)):
           for line in lines[i]:
                pt1 = (line[0],line[1])
                pt2 = (line[2],line[3])
                if (line[1]-line[3])==0 : #print((line[2]-line[3])//(line[0]-line[1]))
                        cv2.line(img, pt1, pt2, (255,255,255), 3) 
                        horizontal_lines.append(line)
    #sort lines to eleminate the closed lines 
    horizontal_lines=sorted(horizontal_lines,key=lambda x: x[1])
    pt=horizontal_lines[0]
    ret_lines.append(pt)
    #loop to retrive the included lines 
    for i in range(1,len(horizontal_lines)):
        if abs(horizontal_lines[i][1]-pt[1])>100:
            #print("included",horizontal_lines[i])
            pt=horizontal_lines[i]
            ret_lines.append(pt)
    return ret_lines

### Get the region of handwritten document: 

In [62]:
def get_region():
    prev_lines=[]
    for i in range(len(training_data)):
        img=training_data[i][0]
        lines=detect_lines(img)
        print(len(lines))
        if len(lines)<3:
            cropped_img=crop_image(img,prev_lines[1][1]+20,prev_lines[2][1]-3,math.floor(prev_lines[1][0]/4),img.shape[1]-1)
            cropped_img=resize_image(cropped_img)
            training_data[i][0]=cropped_img
        elif len(lines)==3:
            cropped_img=crop_image(img,lines[1][1]+10,lines[2][1]-3,math.floor(lines[1][0]/4),img.shape[1]-1)
            cropped_img=resize_image(cropped_img)
            training_data[i][0]=cropped_img
            prev_lines=lines
        cv2.imwrite("../dataset/cropped/crop"+str(i)+".png",cropped_img)

### Horizontal projection: 

In [63]:
def get_histogram(img):
    histogram=[]
    #loop through the image to calcultae the  horizontal histogram of the image
    for i in range(img.shape[0]): #for each row
        sum=0
        for j in range(img.shape[1]): # for each col
            if img[i][j]==0:
                sum+=1
        histogram.append(sum)
    return histogram

In [64]:
#function for ploting
def draw_image_histogram(hist, width,color='k'):
    plt.plot(hist, color=color)
    plt.xlim([0, 256])

### Getting local minima: 

In [65]:
def get_local_min(hist):
    hist=np.array(hist)
    local_minima=argrelextrema(hist, np.less)
    return local_minima

### Lines segmentation: 

In [66]:
def get_lines(img,hist,index):
    prev_y=0
    num=0
    lines=[]
    line_num=0
    for i in range(len(hist)):
        if ((i-prev_y)>0 and (num/(i-prev_y))>0.25 and hist[i]==0 and hist[i-1]!=hist[i] and i-prev_y>100) or ((i-prev_y)>0 and (num/(i-prev_y))>0.25 and hist[i]<20 and hist[i-1]<20 and hist[i-1]!=hist[i] and i-prev_y>100) :
            temp=img[prev_y:i+5,0:img.shape[1]]
            prev_y=i+5
            num=0
            #cv2.imwrite("../dataset/hager/line"+str(index)+str(line_num)+".png",temp)
            line_num+=1
            lines.append(temp)
        elif hist[i]>30:
            num+=1
        
    return lines

In [67]:
 def line_segmentation(img,num):
    im = cv2.erode(img,np.ones((3,200), np.uint8))
    im = cv2.medianBlur(im,5)
    im = cv2.bitwise_not(im)
    w= np.size(img,1)
    # cv2.imshow("i",cv2.resize(im,(700,500)))
    # cv2.waitKey(0)

    lines = []
    _,cnt,_ = cv2.findContours(im,cv2.RETR_EXTERNAL,cv2.CHAIN_APPROX_SIMPLE)
    i=0
    for contour in cnt:
        x=[c[0] for c in contour.reshape(-1,2)]
        y=[c[1] for c in contour.reshape(-1,2)]
        minx,miny,maxx,maxy=np.min(x),np.min(y),np.max(x),np.max(y)
        if (maxx-minx)>=(0.8*w):
            cv2.imwrite("../dataset/reem/line"+str(num)+str(i)+".png",img[miny:maxy,minx:maxx])
            lines.append(img[miny:maxy,minx:maxx])
            i+=1
    return lines

### Scaling the line: 

In [103]:
def max_index(hist):
    max=0
    index=0
    for i in range(len(hist)):
        if hist[i]>max:
            max=hist[i]
            index=i
    return index

In [101]:
def max_value(hist):
    max=0
    for i in range(len(hist)):
        if hist[i]>max:
            max=hist[i]
    return max

In [102]:
def scaling_line(ideal_hist,hist):
    ratio=max_value(ideal_hist)/max_value(hist)
    for i in range(len(hist)):
        hist[i]=hist[i]*ratio
    return hist
    

### Shifting the line: 

In [70]:
""" this function get the index of the lower baseline to shift by this value """
def lower_index(hist,value):
    index=0
    for i in range(len(hist)):
        if abs(hist[i]-value)<20:
            index=i
            break
    return index

In [71]:
#shift right the histo by n 
def shift_right(hist,n):
    e = np.empty_like(hist)
    if n >= 0:
        e[:n] = 0
        e[n:] = hist[:-n]
    else:
        e[n:] = 0
        e[:n] = hist[-n:]
    return e

In [72]:
#shift left the hist by n
def shift_left(hist,n):
    e = np.empty_like(hist)
    x=len(hist)
    if n >= 0:
        e[:x-n] = hist[-(x-n):]
        e[x-n:] = 0
    else:
        e[n:] = 0
        e[:n] = hist[-n:]
    return e

In [73]:
#control the shift direction based on the diff value
def shift(hist,value,ind):
    index=lower_index(hist,value)
    diff=index-ind
    if diff>0:
        hist=shift_left(hist,diff)
    elif diff<0:
        hist=shift_right(hist,-diff)
    return hist

### Calculate the top & bottom line: 

In [74]:
def top_bottom(hist):
    top=-1
    bottom=-1
    for i in range(len(hist)):
        #getting the bottom
        if top!=-1 and hist[i]!=0:
            bottom=i
        #getting the top
        if top==-1 and hist[i]!=0:
            top=i
    return top,bottom

### Calculate the upper & lower baseline: 

In [75]:
#this function calculate the ub & lb by doing exhuasitive search by all possible values
def upper_lower_baseline(ub,lb,top,bottom,hist,ideal_hist):
    prev_err=100000
    ub_hist=-1
    lb_hist=-1
    for u in range(top,len(hist)):
        for l in range(u+1,bottom):
            err=np.power(((ideal_hist[ub]-hist[u])+(ideal_hist[lb]-hist[l])),2)
            if err<prev_err:
                prev_err=err
                ub_hist=u
                lb_hist=l
    return ub_hist,lb_hist        

## Feature Extraction: 

### Calculate from f1 to f6: 

In [76]:
#this function do simple calculation after getting the important data 
def baseline_feature(top,bottom,ub,lb):
    f1=abs(top-ub)
    f2=abs(ub-lb)
    f3=abs(lb-bottom)
    f4=f1/f2
    f5=f1/f3
    f6=f2/f3
    return [f1,f2,f3,f4,f5,f6]

## Classiffiers :

### SVM :

In [94]:
def SVM_fun(x,y,x_test):
    clf = svm.SVC(gamma=0.01,kernel='rbf')
    clf.fit(x, y) 
    y_test=clf.predict(x_test)
    return y_test

### K-NN:

In [88]:
def knn(X,y,test):
    neigh = KNeighborsClassifier(n_neighbors=1)
    neigh.fit(X, y) 
    labels_test=neigh.predict(test)
    return labels_test

# Main Logic:

In [79]:
training_data,test_data,validation_data,training_data_threshold,test_data_threshold=reading_and_preprocess()

Adding document a01-000u to the training data
Adding document a01-003u to the training data
Adding document a01-007u to the training data
Adding document a01-011u to the training data
Adding document a01-014u to the training data
Adding document a01-020u to the training data
Adding document a01-026u to the training data
Adding document a01-030u to the training data
Adding document a01-043u to the training data
Adding document a01-049u to the training data
Adding document r06-041 to the training data
Adding document r06-044 to the training data
Adding document r06-049 to the training data
Adding document r06-053 to the training data
Adding document r06-057 to the training data
Adding document r06-062 to the training data
Adding document r06-066 to the training data
Adding document r06-070 to the training data
Adding document r06-076 to the training data
Adding document r06-035 to the training data
Adding document r06-090 to the training data
Adding document r06-097 to the training data


In [80]:
get_region()

3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3


In [81]:
F=[]
labels=[]
#getting the ideal hist 
img=cv2.imread("../dataset/ideal.png")
#convert to gray scale
img = cv2.cvtColor(img, cv2.COLOR_RGB2GRAY)
ideal_hist=get_histogram(img)

for i in range(len(training_data)):
    img=training_data[i][0]
    #getting the binary image
    ret,img = cv2.threshold(img,127,255,cv2.THRESH_BINARY_INV)
    #calculate the histogram
    hist=get_histogram(img)
    lines=get_lines(img,hist,i)
    for j in range(len(lines)):
        line_hist=get_histogram(lines[j])
        line_hist=scaling_line(ideal_hist,line_hist)
        line_hist=shift(line_hist,352,130)
        top,bottom=top_bottom(line_hist)
        ub,lb=upper_lower_baseline(160,132,top,bottom,line_hist,ideal_hist)
        if ub!=-1 and lb!=-1:
            f=baseline_feature(top,bottom,ub,lb)
            F.append(f)
            labels.append(training_data[i][1])
    print(i,end=" ")

97 183 115 162
0 204 109 148
88 173 101 147
103 145 104 138
102 196 149 172
93 151 113 145
0 180 96 146
0 88 200 95 146
83 181 156 166
93 187 113 146
80 171 102 157
92 188 96 156
0 200 98 150
81 190 117 147
84 164 100 151
0 177 113 147
88 185 101 143
73 186 148 175
1 97 176 109 153
68 137 91 103
92 174 100 153
86 183 104 148
85 173 129 165
62 150 64 105
91 171 96 139
95 189 152 176
85 190 91 143
15 159 103 152
2 72 186 154 169
92 171 121 147
81 193 121 139
3 180 125 135
6 176 137 166
82 182 100 141
1 198 121 154
95 195 120 154
101 160 109 148
84 190 95 139
3 44 167 107 165
93 205 120 140
91 141 109 140
95 188 116 147
87 164 91 140
89 191 124 142
88 181 101 145
100 170 101 143
99 168 103 148
4 96 175 102 149
94 181 148 173
105 169 150 165
96 173 149 170
94 201 125 137
94 175 154 165
92 189 143 188
97 179 120 148
94 159 118 135
88 168 89 151
5 95 192 117 152
0 179 117 141
62 145 71 108
93 188 121 141
93 182 153 174
83 202 105 150
86 176 89 142
73 196 146 172
93 182 131 157
88 177 101 153

In [82]:
training_features=[]
training_labels=[]
test_features=[]
test_labels=[]
for i in range(len(F)):
    if i%10==0:
        test_features.append(F[i])
        test_labels.append(labels[i])
        test_features.append(F[i+1])
        test_labels.append(labels[i+1])
        i+=1
    else:
        training_features.append(F[i])
        training_labels.append(labels[i])
        

In [96]:
ret_labels=knn(training_features,training_labels,test_features)
print(ret_labels)
print(test_labels)
count=0
for i in range(len(ret_labels)):
    if ret_labels[i]==test_labels[i]:
        count+=1
print(count,len(ret_labels),count/len(ret_labels))

['671' '000' '000' '000' '000' '000' '000' '000' '000' '000' '670' '000'
 '000' '000' '000' '000' '670' '000' '670' '000' '670' '670' '000' '670'
 '670' '670' '671' '670' '000' '670' '670' '670' '670' '670' '671' '670'
 '671' '670' '000' '671' '671' '671' '671' '671' '671' '671' '671' '671'
 '670' '671' '671' '671' '671' '671']
['000', '000', '000', '000', '000', '000', '000', '000', '000', '000', '000', '000', '000', '000', '000', '000', '000', '000', '000', '000', '670', '670', '670', '670', '670', '670', '670', '670', '670', '670', '670', '670', '670', '670', '670', '670', '670', '670', '671', '671', '671', '671', '671', '671', '671', '671', '671', '671', '671', '671', '671', '671', '671', '671']
43 54 0.7962962962962963


In [95]:
ret_labels=SVM_fun(training_features,training_labels,test_features)
print(ret_labels)
print(test_labels)
print(len(F))
count=0
for i in range(len(ret_labels)):
    if ret_labels[i]==test_labels[i]:
        count+=1
print(count,len(ret_labels),count/len(ret_labels))

['000' '000' '000' '000' '000' '000' '000' '000' '670' '000' '670' '000'
 '000' '000' '000' '000' '000' '000' '670' '000' '670' '670' '000' '670'
 '670' '670' '670' '670' '000' '670' '670' '670' '670' '670' '671' '670'
 '671' '670' '000' '671' '671' '671' '670' '671' '671' '671' '671' '000'
 '670' '671' '670' '671' '671' '671']
['000', '000', '000', '000', '000', '000', '000', '000', '000', '000', '000', '000', '000', '000', '000', '000', '000', '000', '000', '000', '670', '670', '670', '670', '670', '670', '670', '670', '670', '670', '670', '670', '670', '670', '670', '670', '670', '670', '671', '671', '671', '671', '671', '671', '671', '671', '671', '671', '671', '671', '671', '671', '671', '671']
264
42 54 0.7777777777777778


In [85]:
for i in range(len(F)):
    if labels[i]=='588':
        print(F[i])
print("test")
for i in range(len(ret_labels)):
    if test_labels[i]=='588':
        print(test_features[i])
    
    

test
